In [65]:
from pyspark.sql.functions import *
import random

StatementMeta(sparkpool, 19, 3, Finished, Available, Finished)

In [66]:
from notebookutils import mssparkutils

StatementMeta(sparkpool, 19, 4, Finished, Available, Finished)

In [67]:
dateFormat = "%Y-%m-%d"

import datetime

ts = spark.sql("select current_timestamp() as ctime").collect()[0]["ctime"]

todaydate = ts.strftime(dateFormat)

StatementMeta(sparkpool, 19, 5, Finished, Available, Finished)

In [68]:
sourceaccountname = "storageottonprem011"
sourcelinkedservice = "ls_adls"
apifilelocation = todaydate+".csv"
MovieFilelocation = "data"

StatementMeta(sparkpool, 19, 6, Finished, Available, Finished)

In [69]:
source_full_storage_account_name = "storageottonprem011.dfs.core.windows.net"
spark.conf.set("spark.storage.synapse.linkedServiceName", f"{sourcelinkedservice}")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{source_full_storage_account_name}", "com.microsoft.azure.synapse.tokenlibrary.LinkedServiceBasedTokenProvider")


StatementMeta(sparkpool, 19, 7, Finished, Available, Finished)

In [70]:
MovieFile = f"abfss://refined@{sourceaccountname}.dfs.core.windows.net/{MovieFilelocation}"
apifile = f"abfss://raw@{sourceaccountname}.dfs.core.windows.net/api/{apifilelocation}"
print(MovieFile)
print(apifile)

StatementMeta(sparkpool, 19, 8, Finished, Available, Finished)

abfss://refined@storageottonprem011.dfs.core.windows.net/data
abfss://raw@storageottonprem011.dfs.core.windows.net/api/2024-12-01.csv


In [71]:
try:
    api_df = spark.read.format('csv').option('header','true').option('inferschema','true').load(apifile)
    
    movie_df = spark.read.format('parquet').option('header','true').option('inferschema','true').load(MovieFile)
    
    Genre_df=movie_df.groupBy("Genre") .agg(collect_list(col("Title")).alias("Top_Movies"))
    
    api_df = api_df.withColumn("Preference_genre", split(api_df["Preference_genre"], "\\|"))
    
    updated_api_df = api_df.withColumn("Preference_genre", col("Preference_genre")[0]).select("email", "Preference_genre")
    
    Genre_df = Genre_df.withColumn("movie1", expr("Top_Movies[0]"))\
                    .withColumn("movie2", expr("IF(size(Top_Movies) > 1, Top_Movies[1], NULL)")) \
                    .withColumn("movie3", expr("IF(size(Top_Movies) > 2, Top_Movies[2], NULL)"))\
                    .select("Genre", "movie1", "movie2", "movie3")
    
    result_df = updated_api_df.join(Genre_df, updated_api_df.Preference_genre == Genre_df.Genre).select("email", "movie1", "movie2", "movie3")

    
except Exception as e:
    print(f"File does not exist : {e}")


StatementMeta(sparkpool, 19, 9, Finished, Available, Finished)

In [72]:
destaccountname = "storageottonprem011"
destcontainer = "refined"
destfolder = "data/"

StatementMeta(sparkpool, 19, 10, Finished, Available, Finished)

In [73]:
#writing data to adls
try:
    destpath = f"abfss://{destcontainer}@{destaccountname}.dfs.core.windows.net/movie_recommendation"

    result_df.write\
    .mode("append")\
    .format("parquet")\
    .option("header", "true")\
    .option("Inferschema", "true")\
    .save(destpath)
except Exception as e:
    print("df does not exist")

StatementMeta(sparkpool, 19, 11, Finished, Available, Finished)